In [1]:
from gensim import corpora, models, similarities
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

class StreamCorpus(object):
    def __init__(self, filename):
        self.filename = filename
    def __iter__(self):
        for line in open(self.filename, encoding='UTF-8'):
            yield dictionary.doc2bow(line.lower().split())

def load_stopwords(filename):
    data = ''
    with open(filename, 'r', encoding='UTF-8') as f:
        data = f.read()
    return set(data.split('\n'))

corpus_file = '../Crawler/Data/Corpus.txt'
corpus = StreamCorpus(corpus_file)

dictionary = corpora.Dictionary(line.lower().split() for line in open(corpus_file, encoding='UTF-8'))
stoplist = load_stopwords('../Crawler/Data/Stopwords.txt')
stop_ids = [dictionary.token2id[stopword] for stopword in stoplist
                                          if stopword in dictionary.token2id]
once_ids = [tokenid for tokenid, docfreq in dictionary.dfs.items() if docfreq == 1]
dictionary.filter_tokens(stop_ids + once_ids)
dictionary.filter_extremes()
dictionary.compactify()

dictionary.save('corpus.dict')
corpora.MmCorpus.serialize('corpus.mm', corpus)